- split to construct SEACell based on PC
- and to perform celloracle per sample
- there was mistake (overlapping data between CPTAC and GBMap), so removing CPTAC and keep original one

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import scanpy as sc
import pandas as pd
import numpy as np

In [2]:
os.chdir('../../')

In [12]:
adata = sc.read_h5ad('output/full_filtered.h5ad')
adata

AnnData object with n_obs × n_vars = 1135345 × 5000
    obs: 'author', 'method', 'assay', 'donor_id', 'gs_prediction_main', 'SingleR', 'predicted.annotation_level_3', 'predicted.annotation_level_4.score', 'predicted.annotation_level_4', 'copykat', 'cell_type'
    var: 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'hvg'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [13]:
adata = adata.raw.to_adata()
adata

AnnData object with n_obs × n_vars = 1135345 × 29437
    obs: 'author', 'method', 'assay', 'donor_id', 'gs_prediction_main', 'SingleR', 'predicted.annotation_level_3', 'predicted.annotation_level_4.score', 'predicted.annotation_level_4', 'copykat', 'cell_type'
    var: 'n_cells'
    uns: 'hvg'
    obsm: 'X_pca', 'X_umap'

In [14]:
adata.obs['tumor'] = adata.obs['cell_type'].apply(lambda x: 'tumor' if x == 'malignant cell' else 'normal')

In [15]:
adata.layers['counts'] = adata.X.copy()

In [18]:
samples = adata.obs.donor_id.unique()
samples

['S1T', 'S1V', 'S2T', 'S2V', 'S3T', ..., 'UP-8036', 'UP-8165', 'UP-8167', 'C3N-01334', 'C3N-02769']
Length: 229
Categories (229, object): ['3182', '3282', '3295', 'BT333', ..., 'ndGBM-08', 'ndGBM-09', 'ndGBM-10', 'ndGBM-11']

In [9]:
len(adatas)

229

# 1. original try

In [8]:
# adatas = [adata[adata.obs.donor_id == donor_id] for donor_id in samples]

In [39]:
# i = 0
# for a in adatas:
#     i += 1
#     print('-----------------------------------')
#     print(f'processing {a.obs.donor_id[0]}, {i}/{len(adatas)}')
#     ad = a.copy()
#     ad.raw = ad
#     print('1. filter and normalize')
#     sc.pp.filter_genes(ad, min_cells=1)
#     sc.pp.filter_cells(ad, min_genes=1)
#     sc.pp.normalize_total(ad)
#     sc.pp.log1p(ad)
#     sc.pp.highly_variable_genes(ad, n_top_genes=3000)
#     sc.pp.scale(ad)
    
#     print('2. running PC')
#     sc.tl.pca(ad, use_highly_variable=True)
#     sc.pp.neighbors(ad)
#     sc.tl.umap(ad)

#     print('3. saving')
#     print('\n')
#     ad.write(f'tmp/{ad.obs.donor_id[0]}.h5ad')

-----------------------------------
processing S1T, 1/229
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing S1V, 2/229
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing S2T, 3/229
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing S2V, 4/229
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing S3T, 5/229
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing S3V, 6/229
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing S4T, 7/229
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing S4V, 8/229
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing S5T, 9/229
1. filter and normalize
2. running PC
3. saving


----------------------------

# 2. redo CPTAC

- remove CPTAC dataset and only keep one from GBmap

In [28]:
a1 = adata.obs[adata.obs.author == "Wang2021"].donor_id.unique()
a2 = adata.obs[adata.obs.author == "cptac"].donor_id.unique()
keep = set(a1) | set(a2)

In [38]:
files_to_remove = [os.path.join('tmp', f) for f in os.listdir('tmp') if f.endswith('.h5ad') and f[:-5] in keep]
os.system(f'rm {" ".join(files_to_remove)}')

0

- then re-run

In [40]:
bdata = adata[adata.obs.author == "Wang2021"].copy()
bdata

AnnData object with n_obs × n_vars = 154099 × 29437
    obs: 'author', 'method', 'assay', 'donor_id', 'gs_prediction_main', 'SingleR', 'predicted.annotation_level_3', 'predicted.annotation_level_4.score', 'predicted.annotation_level_4', 'copykat', 'cell_type', 'tumor'
    var: 'n_cells'
    uns: 'hvg'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [42]:
samples = bdata.obs.donor_id.unique()
len(samples)

18

In [43]:
adatas = [adata[adata.obs.donor_id == donor_id] for donor_id in samples]

In [45]:
i = 0
for a in adatas:
    i += 1
    print('-----------------------------------')
    print(f'processing {a.obs.donor_id[0]}, {i}/{len(adatas)}')
    ad = a.copy()
    ad.raw = ad
    print('1. filter and normalize')
    sc.pp.filter_genes(ad, min_cells=1)
    sc.pp.filter_cells(ad, min_genes=1)
    sc.pp.normalize_total(ad)
    sc.pp.log1p(ad)
    sc.pp.highly_variable_genes(ad, n_top_genes=3000)
    sc.pp.scale(ad)
    
    print('2. running PC')
    sc.tl.pca(ad, use_highly_variable=True)
    sc.pp.neighbors(ad)
    sc.tl.umap(ad)

    print('3. saving')
    print('\n')
    ad.write(f'tmp/{ad.obs.donor_id[0]}.h5ad')

-----------------------------------
processing C3L-02705, 1/18
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing C3L-03405, 2/18
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing C3L-03968, 3/18
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing C3N-00662, 4/18
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing C3N-01334, 5/18
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing C3N-01798, 6/18
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing C3N-01814, 7/18
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing C3N-01815, 8/18
1. filter and normalize
2. running PC
3. saving


-----------------------------------
processing C3N-01816, 9/18
1. filter and normalize
2. runnin

In [ ]:
with open('tmp/cptac_rerun.txt', 'w') as f:
    f.write('\n'.join(keep))